### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_random

CONCEPT_NAME = concepts.concept_random.__name__


2023-10-25 18:44:10.496624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [4]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

Positive cases: 2544it [02:16, 14.33it/s]                          

Positive cases: 2545it [02:30, 14.33it/s]

In [5]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2424, 4, 5, 5)


In [ ]:
# Find how many positive cases are identical to other positive cases and count the number of toatal cases
identical_cases = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            identical_cases += 1

print(f"Total cases: {len(positive_cases)}")
print(f"Identical cases: {identical_cases}")

In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [6]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0])
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Positions to consider: 2000


Getting resblock outputs: 100%|██████████| 154/154 [00:00<00:00, 252.97it/s]


Performing regression for layer 0
Epoch 1/50
63/63 [==============================] - 1s 4ms/step - loss: 1.0028 - val_loss: 0.9863
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9763 - val_loss: 0.9617
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9517 - val_loss: 0.9390
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9288 - val_loss: 0.9168
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9069 - val_loss: 0.8959
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8862 - val_loss: 0.8759
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8665 - val_loss: 0.8571
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8479 - val_loss: 0.8392
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8304 - val_loss: 0.8228
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8142 - va

Getting resblock outputs: 100%|██████████| 154/154 [00:00<00:00, 275.42it/s]


Performing regression for layer 0
Epoch 1/50
63/63 [==============================] - 1s 4ms/step - loss: 1.0108 - val_loss: 0.9863
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9835 - val_loss: 0.9729
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9695 - val_loss: 0.9609
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9563 - val_loss: 0.9487
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9436 - val_loss: 0.9372
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9318 - val_loss: 0.9258
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9201 - val_loss: 0.9149
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9088 - val_loss: 0.9043
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8977 - val_loss: 0.8938
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8875 - va

...

Performing regression for layer 0
Epoch 1/50
63/63 [==============================] - 1s 4ms/step - loss: 1.0211 - val_loss: 1.0119
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 1.0015 - val_loss: 0.9963
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9860 - val_loss: 0.9829
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9714 - val_loss: 0.9699
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9585 - val_loss: 0.9581
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9465 - val_loss: 0.9464
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9349 - val_loss: 0.9357
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9237 - val_loss: 0.9255
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9138 - val_loss: 0.9159
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9031 - va

Getting resblock outputs: 100%|██████████| 154/154 [00:00<00:00, 240.37it/s]


Performing regression for layer 0
Epoch 1/50
63/63 [==============================] - 1s 4ms/step - loss: 1.0088 - val_loss: 1.0026
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9920 - val_loss: 0.9888
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9775 - val_loss: 0.9760
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9649 - val_loss: 0.9645
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9523 - val_loss: 0.9530
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9409 - val_loss: 0.9422
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9295 - val_loss: 0.9322
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9190 - val_loss: 0.9221
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9086 - val_loss: 0.9120
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8982 - va

Getting resblock outputs: 100%|██████████| 154/154 [00:00<00:00, 282.99it/s]


Performing regression for layer 0
Epoch 1/50
63/63 [==============================] - 1s 3ms/step - loss: 1.0119 - val_loss: 1.0064
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9922 - val_loss: 0.9899
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9763 - val_loss: 0.9765
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9630 - val_loss: 0.9642
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9510 - val_loss: 0.9527
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9400 - val_loss: 0.9419
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9291 - val_loss: 0.9321
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9191 - val_loss: 0.9221
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.9094 - val_loss: 0.9127
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.8998 - va